In [3]:
# 查看当前挂载的数据集目录
!ls /home/aistudio/data/

data7983


In [4]:
# 查看个人持久化工作区文件
!ls /home/aistudio/work/

pretrained


!unzip data/data7983/cat_12.zip

In [3]:
!wget http://paddle-imagenet-models-name.bj.bcebos.com/ResNet101_pretrained.tar

--2019-06-17 11:42:16--  http://paddle-imagenet-models-name.bj.bcebos.com/ResNet101_pretrained.tar
Resolving paddle-imagenet-models-name.bj.bcebos.com (paddle-imagenet-models-name.bj.bcebos.com)... 220.181.33.48, 220.181.33.43, 220.181.33.44, ...
Connecting to paddle-imagenet-models-name.bj.bcebos.com (paddle-imagenet-models-name.bj.bcebos.com)|220.181.33.48|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 179148800 (171M) [application/x-tar]
Saving to: ‘ResNet101_pretrained.tar’

ResNet101_pretraine 100%[===================>] 170.85M  20.4MB/s    in 6.0s    

2019-06-17 11:42:23 (28.7 MB/s) - ‘ResNet101_pretrained.tar’ saved [179148800/179148800]



In [4]:
!tar -xvf ResNet101_pretrained.tar -C work/pretrained

ResNet101_pretrained/
ResNet101_pretrained/res4b19_branch2b_weights
ResNet101_pretrained/bn5a_branch2b_scale
ResNet101_pretrained/bn4b8_branch2c_mean
ResNet101_pretrained/bn4b16_branch2b_mean
ResNet101_pretrained/bn4b1_branch2b_scale
ResNet101_pretrained/bn3b_branch2c_offset
ResNet101_pretrained/bn4b2_branch2b_variance
ResNet101_pretrained/res4b1_branch2c_weights
ResNet101_pretrained/bn4b1_branch2c_variance
ResNet101_pretrained/bn4b22_branch2c_variance
ResNet101_pretrained/bn5c_branch2b_variance
ResNet101_pretrained/bn3a_branch2a_variance
ResNet101_pretrained/res4b5_branch2b_weights
ResNet101_pretrained/bn4b5_branch2b_scale
ResNet101_pretrained/bn4b2_branch2b_scale
ResNet101_pretrained/bn3a_branch1_offset
ResNet101_pretrained/bn4b8_branch2b_offset
ResNet101_pretrained/bn5a_branch2c_variance
ResNet101_pretrained/bn4b18_branch2a_offset
ResNet101_pretrained/bn4b20_branch2b_offset
ResNet101_pretrained/bn3c_branch2a_variance
ResNet101_pretrained/bn3b_branch2b_mean
ResNet101_pretrained/res4b

# Step-1

In [1]:
import os
import shutil
import paddle as paddle
import paddle.fluid as fluid
from paddle.fluid.param_attr import ParamAttr
import reader
import numpy as np
import pandas as pd

In [2]:
##生成test.txt
#filepath='cat_12/cat_12_test'
#pathlist=os.listdir(filepath)
#print(pathlist)
#with open('cat_12/test_list.txt','w+') as f:
    #for file_name in pathlist:
        #f.write(file_name+'\n')

In [3]:
a=pd.read_table('cat_12/train_list.txt',header=None)
print(a[1].value_counts())

11    180
9     180
7     180
5     180
3     180
1     180
10    180
8     180
6     180
4     180
2     180
0     180
Name: 1, dtype: int64


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [4]:
##处理数据集，将原有训练集按二八比例分割
with open('cat_12/train_list.txt','r') as f:
    with open('cat_12/train_split_list.txt','w+') as train:
        lines=f.readlines()
    with open('cat_12/train_split_list.txt','w+') as train:
        for i in range(0,2160,180):
            for line in lines[i:i+135]:
                train.write(line)
    with open('cat_12/val_split_list.txt','w+') as val:
        for i in range(0,2160,180):
            for line in lines[i+135:i+180]:
                val.write(line)

In [5]:
train=pd.read_table('cat_12/train_split_list.txt',header=None)
print(train[1].value_counts())
val=pd.read_table('cat_12/val_split_list.txt',header=None)
print(val[1].value_counts())

11    135
10    135
9     135
8     135
7     135
6     135
5     135
4     135
3     135
2     135
1     135
0     135
Name: 1, dtype: int64
11    45
10    45
9     45
8     45
7     45
6     45
5     45
4     45
3     45
2     45
1     45
0     45
Name: 1, dtype: int64


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
##获取猫数据
train_reader = paddle.batch(reader.train(), batch_size=16)
test_reader = paddle.batch(reader.val(), batch_size=16)

In [7]:
##定义残差网络
def resnet(input):
    def conv_bn_layer(input, num_filters, filter_size, stride=1, groups=1, act=None, name=None):
        conv = fluid.layers.conv2d(input=input,
                                   num_filters=num_filters,
                                   filter_size=filter_size,
                                   stride=stride,
                                   padding=(filter_size - 1) // 2,
                                   groups=groups,
                                   act=None,
                                   param_attr=ParamAttr(name=name + "_weights"),
                                   bias_attr=False,
                                   name=name + '.conv2d.output.1')
        if name == "conv1":
            bn_name = "bn_" + name
        else:
            bn_name = "bn" + name[3:]
        return fluid.layers.batch_norm(input=conv,
                                       act=act,
                                       name=bn_name + '.output.1',
                                       param_attr=ParamAttr(name=bn_name + '_scale'),
                                       bias_attr=ParamAttr(bn_name + '_offset'),
                                       moving_mean_name=bn_name + '_mean',
                                       moving_variance_name=bn_name + '_variance', )

    def shortcut(input, ch_out, stride, name):
        ch_in = input.shape[1]
        if ch_in != ch_out or stride != 1:
            return conv_bn_layer(input, ch_out, 1, stride, name=name)
        else:
            return input
            
    def bottleneck_block(input, num_filters, stride, name):
        conv0 = conv_bn_layer(input=input,
                              num_filters=num_filters,
                              filter_size=1,
                              act='relu',
                              name=name + "_branch2a")
        conv1 = conv_bn_layer(input=conv0,
                              num_filters=num_filters,
                              filter_size=3,
                              stride=stride,
                              act='relu',
                              name=name + "_branch2b")
        conv2 = conv_bn_layer(input=conv1,
                              num_filters=num_filters * 4,
                              filter_size=1,
                              act=None,
                              name=name + "_branch2c")

        short = shortcut(input, num_filters * 4, stride, name=name + "_branch1")

        return fluid.layers.elementwise_add(x=short, y=conv2, act='relu', name=name + ".add.output.5")

    depth = [3, 4, 23, 3]
    num_filters = [64, 128, 256, 512]

    conv = conv_bn_layer(input=input, num_filters=64, filter_size=7, stride=2, act='relu', name="conv1")
    conv = fluid.layers.pool2d(input=conv, pool_size=3, pool_stride=2, pool_padding=1, pool_type='max')

    for block in range(len(depth)):
        for i in range(depth[block]):
            if block == 2:
                if i == 0:
                    conv_name="res"+str(block+2)+"a"
                else:
                    conv_name="res"+str(block+2)+"b"+str(i)
            else:
                    conv_name="res"+str(block+2)+chr(97+i)
            conv = bottleneck_block(input=conv,
                                    num_filters=num_filters[block],
                                    stride=2 if i == 0 and block != 0 else 1,
                                    name=conv_name)

    pool = fluid.layers.pool2d(input=conv, pool_size=7, pool_type='avg', global_pooling=True)
    return pool


In [8]:
##定义输入层
image=fluid.layers.data(name='image',shape=[3,224,224],dtype='float32')
label=fluid.layers.data(name='label',shape=[1],dtype='int64')

In [9]:
##停止梯度下降
pool=resnet(image)
pool.stop_gradient=True

##创建主程序
base_model_program=fluid.default_main_program().clone()
model=fluid.layers.fc(input=pool,size=12,act='softmax')

In [10]:
##定义损失函数和准确率函数
cost=fluid.layers.cross_entropy(input=model,label=label)
avg_cost=fluid.layers.mean(cost)
acc=fluid.layers.accuracy(input=model,label=label)

##定义优化方法
optimizer=fluid.optimizer.AdamOptimizer(learning_rate=5e-4)
opts=optimizer.minimize(avg_cost)

##定义训练场所
use_gpu=True
place=fluid.CUDAPlace(0) if use_gpu else fluid.CPUPlace()
exe=fluid.Executor(place)

##进行参数初始化
exe.run(fluid.default_startup_program())

[]

In [11]:
##预训练模型路径
src_pretrain_model_path='work/pretrained/ResNet101_pretrained'

##判断模型文件是否存在
def if_exit(var):
    path=os.path.join(src_pretrain_model_path,var.name)
    exist=os.path.exists(path)
    return exist

##加载模型文件，且只加载存在模型的模型文件
fluid.io.load_vars(executor=exe,dirname=src_pretrain_model_path,predicate=if_exit,main_program=base_model_program)

In [12]:
##优化内存
optimized = fluid.transpiler.memory_optimize(input_program=fluid.default_main_program(), print_log=False)

##定义数据维度
feeder=fluid.DataFeeder(place=place,feed_list=[image,label])

for pass_id in range(15):
    for batch_id,data in enumerate(train_reader()):
        train_cost,train_acc=exe.run(program = fluid.default_main_program(),feed=feeder.feed(data),fetch_list=[avg_cost,acc])
        if batch_id%50==0:
            print('Pass:%d, Batch:%d, Cost:%0.5f, Accuracy:%0.5f' %
                  (pass_id, batch_id, train_cost[0], train_acc[0]))

memory_optimize is deprecated. Use CompiledProgram and Executor


Pass:0, Batch:0, Cost:2.51998, Accuracy:0.12500
Pass:0, Batch:50, Cost:1.77459, Accuracy:0.50000
Pass:0, Batch:100, Cost:1.34139, Accuracy:0.68750
Pass:1, Batch:0, Cost:1.36643, Accuracy:0.62500
Pass:1, Batch:50, Cost:1.15108, Accuracy:0.75000
Pass:1, Batch:100, Cost:1.31792, Accuracy:0.56250
Pass:2, Batch:0, Cost:0.89522, Accuracy:0.81250
Pass:2, Batch:50, Cost:0.70764, Accuracy:0.87500
Pass:2, Batch:100, Cost:0.80643, Accuracy:0.68750
Pass:3, Batch:0, Cost:0.59440, Accuracy:0.81250
Pass:3, Batch:50, Cost:0.66792, Accuracy:0.81250
Pass:3, Batch:100, Cost:1.13903, Accuracy:0.62500
Pass:4, Batch:0, Cost:0.89670, Accuracy:0.75000
Pass:4, Batch:50, Cost:0.41714, Accuracy:0.93750
Pass:4, Batch:100, Cost:0.81609, Accuracy:0.81250
Pass:5, Batch:0, Cost:0.58426, Accuracy:0.87500
Pass:5, Batch:50, Cost:0.60263, Accuracy:0.75000
Pass:5, Batch:100, Cost:0.58216, Accuracy:0.87500
Pass:6, Batch:0, Cost:0.92189, Accuracy:0.68750
Pass:6, Batch:50, Cost:0.86046, Accuracy:0.68750
Pass:6, Batch:100, Co

In [13]:
save_pretrain_model_path='models/step-1_model/'

##删除旧的模型文件
shutil.rmtree(save_pretrain_model_path,ignore_errors=True)
##创建保存模型文件记录
os.makedirs(save_pretrain_model_path)
##保存参数模型
fluid.io.save_params(executor=exe,dirname=save_pretrain_model_path)

# Step-2

In [1]:
import os
import shutil
import paddle as paddle
import paddle.dataset.flowers as flowers
import paddle.fluid as fluid
from paddle.fluid.param_attr import ParamAttr
import reader

In [2]:
##获取数据
train_reader = paddle.batch(reader.train(), batch_size=16)
test_reader = paddle.batch(reader.val(), batch_size=16)

In [3]:
##定义resnet，加上fc
def resnet(input,class_dim):
    def conv_bn_layer(input, num_filters, filter_size, stride=1, groups=1, act=None, name=None):
        conv = fluid.layers.conv2d(input=input,
                                   num_filters=num_filters,
                                   filter_size=filter_size,
                                   stride=stride,
                                   padding=(filter_size - 1) // 2,
                                   groups=groups,
                                   act=None,
                                   param_attr=ParamAttr(name=name + "_weights"),
                                   bias_attr=False,
                                   name=name + '.conv2d.output.1')
        if name == "conv1":
            bn_name = "bn_" + name
        else:
            bn_name = "bn" + name[3:]
        return fluid.layers.batch_norm(input=conv,
                                       act=act,
                                       name=bn_name + '.output.1',
                                       param_attr=ParamAttr(name=bn_name + '_scale'),
                                       bias_attr=ParamAttr(bn_name + '_offset'),
                                       moving_mean_name=bn_name + '_mean',
                                       moving_variance_name=bn_name + '_variance', )

    def shortcut(input, ch_out, stride, name):
        ch_in = input.shape[1]
        if ch_in != ch_out or stride != 1:
            return conv_bn_layer(input, ch_out, 1, stride, name=name)
        else:
            return input
            
    def bottleneck_block(input, num_filters, stride, name):
        conv0 = conv_bn_layer(input=input,
                              num_filters=num_filters,
                              filter_size=1,
                              act='relu',
                              name=name + "_branch2a")
        conv1 = conv_bn_layer(input=conv0,
                              num_filters=num_filters,
                              filter_size=3,
                              stride=stride,
                              act='relu',
                              name=name + "_branch2b")
        conv2 = conv_bn_layer(input=conv1,
                              num_filters=num_filters * 4,
                              filter_size=1,
                              act=None,
                              name=name + "_branch2c")

        short = shortcut(input, num_filters * 4, stride, name=name + "_branch1")

        return fluid.layers.elementwise_add(x=short, y=conv2, act='relu', name=name + ".add.output.5")

    depth = [3, 4, 23, 3]
    num_filters = [64, 128, 256, 512]

    conv = conv_bn_layer(input=input, num_filters=64, filter_size=7, stride=2, act='relu', name="conv1")
    conv = fluid.layers.pool2d(input=conv, pool_size=3, pool_stride=2, pool_padding=1, pool_type='max')

    for block in range(len(depth)):
        for i in range(depth[block]):
            if block == 2:
                if i == 0:
                    conv_name="res"+str(block+2)+"a"
                else:
                    conv_name="res"+str(block+2)+"b"+str(i)
            else:
                    conv_name="res"+str(block+2)+chr(97+i)
            conv = bottleneck_block(input=conv,
                                    num_filters=num_filters[block],
                                    stride=2 if i == 0 and block != 0 else 1,
                                    name=conv_name)

    pool = fluid.layers.pool2d(input=conv, pool_size=7, pool_type='avg', global_pooling=True)
    output=fluid.layers.fc(input=pool,size=class_dim,act='softmax')
    return output

In [4]:
##定义输入层
image = fluid.layers.data(name='image', shape=[3, 224, 224], dtype='float32')
label = fluid.layers.data(name='label', shape=[1], dtype='int64')

##获取分类器
model = resnet(image,12)

In [5]:
##获取损失函数和准确率函数
cost = fluid.layers.cross_entropy(input=model, label=label)
avg_cost = fluid.layers.mean(cost)
acc = fluid.layers.accuracy(input=model, label=label)

In [6]:
##获取训练和测试程序
test_program = fluid.default_main_program().clone(for_test=True)

In [7]:
##定义优化方法
optimizer=fluid.optimizer.AdamOptimizer(learning_rate=5e-4)
opts=optimizer.minimize(avg_cost)

In [8]:
##定义一个使用GPU的执行器
use_gpu=True
place=fluid.CUDAPlace(0) if use_gpu else fluid.CPUPlace()
exe=fluid.Executor(place)

##进行参数初始化
exe.run(fluid.default_startup_program())

[]

In [9]:
##经过step-1处理后的的预训练模型
pretrained_model_path = 'models/step-1_model/'

##加载经过处理的模型
fluid.io.load_params(executor=exe, dirname=pretrained_model_path)

In [10]:
##定义数据维度
feeder=fluid.DataFeeder(place=place,feed_list=[image,label])
for pass_id in range(10):
    ##训练
    for batch_id,data in enumerate(train_reader()):
        train_cost,train_acc=exe.run(program=fluid.default_main_program(),feed=feeder.feed(data),fetch_list=[avg_cost,acc])
        if batch_id%10==0:
            print('Pass:%d, Batch:%d, Cost:%0.5f, Accuracy:%0.5f' %
                  (pass_id, batch_id, train_cost[0], train_acc[0]))
    ##测试
    test_accs=[]
    test_costs=[]
    for batch_id,data in enumerate(test_reader()):
        test_cost,test_acc=exe.run(program=test_program,feed=feeder.feed(data), fetch_list=[avg_cost,acc])
        test_accs.append(test_acc[0])
        test_costs.append(test_cost[0])
    test_cost = (sum(test_costs) / len(test_costs))
    test_acc = (sum(test_accs) / len(test_accs))
    print('Test:%d, Cost:%0.5f, Accuracy:%0.5f' % (pass_id, test_cost, test_acc))
    

Pass:0, Batch:0, Cost:0.46568, Accuracy:0.81250
Pass:0, Batch:10, Cost:3.46601, Accuracy:0.18750
Pass:0, Batch:20, Cost:1.98238, Accuracy:0.25000
Pass:0, Batch:30, Cost:1.77926, Accuracy:0.37500
Pass:0, Batch:40, Cost:2.02586, Accuracy:0.50000
Pass:0, Batch:50, Cost:1.21160, Accuracy:0.68750
Pass:0, Batch:60, Cost:2.23126, Accuracy:0.25000
Pass:0, Batch:70, Cost:1.66542, Accuracy:0.56250
Pass:0, Batch:80, Cost:1.27224, Accuracy:0.56250
Pass:0, Batch:90, Cost:1.24417, Accuracy:0.50000
Pass:0, Batch:100, Cost:1.53577, Accuracy:0.56250
Test:0, Cost:1.35709, Accuracy:0.55025
Pass:1, Batch:0, Cost:1.68541, Accuracy:0.43750
Pass:1, Batch:10, Cost:1.51056, Accuracy:0.37500
Pass:1, Batch:20, Cost:1.39097, Accuracy:0.62500
Pass:1, Batch:30, Cost:0.99089, Accuracy:0.68750
Pass:1, Batch:40, Cost:0.96091, Accuracy:0.68750
Pass:1, Batch:50, Cost:1.12623, Accuracy:0.68750
Pass:1, Batch:60, Cost:1.08348, Accuracy:0.56250
Pass:1, Batch:70, Cost:1.38928, Accuracy:0.50000
Pass:1, Batch:80, Cost:0.74853,

In [12]:
##保存预测模型
save_path = 'models/step_2_model/'

##删除旧的模型文件
shutil.rmtree(save_path, ignore_errors=True)
##创建保持模型文件目录
os.makedirs(save_path)
##保存预测模型
fluid.io.save_inference_model(save_path, feeded_var_names=[image.name], target_vars=[model], executor=exe)

['save_infer_model/scale_0']

In [13]:
# 在生成模型文件（model）后，运行该cell
# 在生成CSV文件过程中，有任何问题可咨询助教
# 助教将辅助大家生成CSV文件


# 注意！！！
# 除变量"SAVE_DIRNAME"外，请不要私自改动其他代码。有问题请联系助教
#########################################################################
"""注意！！！

请将变量"SAVE_DIRNAME"的值修改为模型文件（model）的路径。如：
SAVE_DIRNAME = './models/step_2_model'
"""
# 请对该行代码出修改：
SAVE_DIRNAME = './models/step_2_model'  # 保存好的 inference model 的路径

#########################################################################


"""
运行generate_CSV_file_with_infer.py
脚本文件generate_CSV_file_with_infer.py实现生成CSV文件的功能
脚本文件将读取训练好的模型（model）和测试集数据（test）
并将模型对测试集数据的预测结果保存为CSV文件
"""

# coding:utf-8
from __future__ import print_function
import os
import json

import paddle
import paddle.fluid as fluid
import numpy as np
from PIL import Image
import sys


# 根据竞赛规则更改 TopK 的值。本次竞赛只看 top 1 的值
TOP_K = 1

SAVE_DIRNAME = './models/step_2_model'  # 保存好的 inference model 的路径

DATA_DIM = 224

use_cuda = True
place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()
exe = fluid.Executor(place)

# 下面行代码根据自己保存时的写法匹配
[inference_program, feed_target_names, fetch_targets] = fluid.io.load_inference_model( SAVE_DIRNAME, exe,
                # model_filename='model',
                # params_filename='params'
                # model_filename = 'fc_0.w_0',
                # params_filename = 'params'
                )


def real_infer_one_img(im):
    infer_result = exe.run(
        inference_program,
        feed={feed_target_names[0]: im},
        fetch_list=fetch_targets)

    # print(infer_result)
    # 打印预测结果
    mini_batch_result = np.argsort(infer_result)  # 找出可能性最大的列标，升序排列
    # print(mini_batch_result.shape)
    mini_batch_result = mini_batch_result[0][:, -TOP_K:]  # 把这些列标拿出来
    # print('预测结果：%s' % mini_batch_result)
    # 打印真实结果
    # label = np.array(test_y)  # 转化为 label
    # print('真实结果：%s' % label)
    mini_batch_result = mini_batch_result.flatten() #拉平了，只吐出一个 array
    mini_batch_result = mini_batch_result[::-1] #逆序
    return mini_batch_result


def resize_short(img, target_size):
    percent = float(target_size) / min(img.size[0], img.size[1])
    resized_width = int(round(img.size[0] * percent))
    resized_height = int(round(img.size[1] * percent))
    img = img.resize((resized_width, resized_height), Image.LANCZOS)
    return img


def crop_image(img, target_size, center):
    width, height = img.size
    size = target_size
    if center == True:
        w_start = (width - size) / 2
        h_start = (height - size) / 2
    else:
        w_start = np.random.randint(0, width - size + 1)
        h_start = np.random.randint(0, height - size + 1)
    w_end = w_start + size
    h_end = h_start + size
    img = img.crop((w_start, h_start, w_end, h_end))
    return img

img_mean = np.array([0.485, 0.456, 0.406]).reshape((3, 1, 1))
img_std = np.array([0.229, 0.224, 0.225]).reshape((3, 1, 1))

def process_image(img_path):
    img = Image.open(img_path)
    img = resize_short(img, target_size=256)
    img = crop_image(img, target_size=DATA_DIM, center=True)

    if img.mode != 'RGB':
        img = img.convert('RGB')

    img = np.array(img).astype(np.float32).transpose((2, 0, 1)) / 255
    img -= img_mean
    img /= img_std

    img = np.expand_dims(img, axis=0)
    return img


def convert_list(my_list):
    my_list = list(my_list)
    my_list = map(lambda x:str(x), my_list)
    # print('_'.join(my_list))
    return '_'.join(my_list)


def infer(file_path):
    im = process_image(file_path)
    result = real_infer_one_img(im)
    result = convert_list(result)
    return result




def createCSVFile(cat_12_test_path):
    lines = []

    # 获取所有的文件名
    img_paths = os.listdir(cat_12_test_path)
    for file_name in img_paths:
        file_name = file_name
        file_abs_path = os.path.join(cat_12_test_path, file_name)
        result_classes = infer(file_abs_path)

        file_predict_classes = result_classes

        line = '%s,%s\n'%(file_name, file_predict_classes)
        lines.append(line)

    with open('csv_file_path.csv', 'w') as f:
        f.writelines(lines)


abs_path = r'./cat_12/cat_12_test' # cat_12_test 文件夹的真实路径
createCSVFile(abs_path)